# <font color = 'red'> ***CHATBOT FOR LOAN ASSISSTANCE***

# <font color = 'teal'> ***Installing Necessary Libraries***

In [52]:
!pip install langchain openai chromadb tiktoken langchain-community pypdf docarray docx2txt gradio

# <font color='teal'> ***Connecting the G-Drive***

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# <font color='teal'> ***Importing Libraries***

In [54]:
import os
import docx2txt
from IPython.display import Markdown, display
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA, StuffDocumentsChain, ConversationalRetrievalChain
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.utilities import GoogleSearchAPIWrapper
import gradio as gr

# <font color='teal'> ***Setting the OpenAPI***

In [ ]:
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# <font color='teal'> ***Loading the Data***

In [56]:
def load_data_from_directory(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            if filename.endswith(".txt"):
                loader = TextLoader(file_path)
                documents.extend(loader.load())
            elif filename.endswith(".pdf"):
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
            elif filename.endswith(".docx") or filename.endswith(".docx"):
                loader = Docx2txtLoader(file_path)
                documents.extend(loader.load())
    return documents

In [ ]:
directory_path = 'Model_ChatBot/ChatBot/Text_Data'
documents = load_data_from_directory(directory_path)

# <font color='teal'> **Splitting the data into chunks**

In [58]:
# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# <font color='teal'> ***Document Processing***

In [72]:
# Process the data into a standard format
def process_data(docs):
    processed_data = []
    for doc in docs:
      # extracting key information
        key_info = {
            "page_content": doc.page_content[:200],
            "source": doc.metadata.get("source", "Unknown"),
            "metadata": doc.metadata
          }
        processed_data.append(key_info)
    return processed_data

processed_documents = process_data(texts)

In [76]:
for i, doc in enumerate(processed_documents):
    print(f"Document {i+1}:")
    print(f"  Page Content (truncated to 200 chars): {doc['page_content'][:200]}...")
    print(f"  Source: {doc['source']}")
    print(f"  Metadata: {doc['metadata']}")
    print("-" * 20)

Document 1:
  Page Content (truncated to 200 chars): source: https://www.inderscienceonline.com/doi/abs/10.1504/IJICBM.2013.053102?journalCode=ijicbm...
  Source: /content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_3.txt
  Metadata: {'source': '/content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_3.txt'}
--------------------
Document 2:
  Page Content (truncated to 200 chars): Skip to main content
Site Logo, redirects to the Home Page
Anywhere
Quick Search anywhere
Enter words / phrases / DOI / ISSN / keywords / authors etc
Enter words / phrases / DOI / ISSN / keywords / au...
  Source: /content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_3.txt
  Metadata: {'source': '/content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_3.txt'}
--------------------
Document 3:
  Page Content (truncated to 200 chars): Keywords
loan, bank, recovery agents, defaulter, qualitative, India
References
1. Acharya, V.V. , Bharath, S.T. , S

# <font color='teal'> ***Creating Vectorstore***

In [78]:
# Create embeddings and vectorstore
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

# <font color='teal'> ***Loading the Model***

In [60]:
# Load the Model
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")

# <font color='teal'>  ***Creating Memory Buffer***

In [61]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# <font color='teal'> ***Retriveing Relevant Data***

In [62]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    memory=memory
)

# <font color='teal'> ***Testing the chatbot***

In [63]:
# Create the QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
)

# <font color='teal'> **Giving Prompts**

In [68]:
# Get user query
user_query = "How has the COVID-19 pandemic influenced government financing strategies for small and medium enterprises (SMEs)?" #@param {type:"string"}

In [69]:
if user_query:
    result = qa({"query": user_query})
    display(Markdown(f"### **Answer:** {result['result']}"))
    # Showing source documents
    display(Markdown("### **Source Documents:**"))
    for doc in result['source_documents']:
        source = doc.metadata['source']
        content = doc.page_content
        display(Markdown(f"- **Source**: {source}\n  **Content**: {content}"))
else:
    print("Please enter a query.")

### **Answer:** The text doesn't provide specific information on the necessities to get a loan.

### **Source Documents:**

- **Source**: /content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_4.pdf
  **Content**: banks might deny a student a loan if he or she is applying to "ordinary degree-
granting universities."
For more information on paying off a student loan, take a look at these related 
posts:
What Do We Know About Education Crowdfunding?
India’s Youth Is Actively Making A Difference - Here’s How
Business Loans
While undertaking a business loan, if the lender is not able to recover the loan 
amount, then the borrower’s business may have to file for bankruptcy. Secondly, 
long-term loans may usually only be granted if your organisation has a stable 
and consistent cash flow. As a result, if you're a small-to-medium-sized 
business with irregular cash flow, such a loan may be tough to obtain.
Further reading:
Got A Great Idea For Your Start-Up But No Funding? Crowdfunding Can Be Your 
Solution!
Steps To Pay Off Loans & Debts
Dealing with debt and loans to help pay bills can be difficult, but you don't 
have to go through it alone. Here are a few steps you can take to repay what you 
owe in a sustainable manner.
Note Down How Much You Owe
Make a list of all of your debts, including the creditor, the total amount owed, 
the monthly payment, the interest rate, and the due date. Your credit report can 
help you validate the debts on your list. With all of your bills in front of 
you, you'll be able to see the broad picture and keep on top of your entire debt 
situation.
Prioritize Your Debts
Not all debts are created equal when it comes to making the initial payment.

- **Source**: /content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_4.pdf
  **Content**: banks might deny a student a loan if he or she is applying to "ordinary degree-
granting universities."
For more information on paying off a student loan, take a look at these related 
posts:
What Do We Know About Education Crowdfunding?
India’s Youth Is Actively Making A Difference - Here’s How
Business Loans
While undertaking a business loan, if the lender is not able to recover the loan 
amount, then the borrower’s business may have to file for bankruptcy. Secondly, 
long-term loans may usually only be granted if your organisation has a stable 
and consistent cash flow. As a result, if you're a small-to-medium-sized 
business with irregular cash flow, such a loan may be tough to obtain.
Further reading:
Got A Great Idea For Your Start-Up But No Funding? Crowdfunding Can Be Your 
Solution!
Steps To Pay Off Loans & Debts
Dealing with debt and loans to help pay bills can be difficult, but you don't 
have to go through it alone. Here are a few steps you can take to repay what you 
owe in a sustainable manner.
Note Down How Much You Owe
Make a list of all of your debts, including the creditor, the total amount owed, 
the monthly payment, the interest rate, and the due date. Your credit report can 
help you validate the debts on your list. With all of your bills in front of 
you, you'll be able to see the broad picture and keep on top of your entire debt 
situation.
Prioritize Your Debts
Not all debts are created equal when it comes to making the initial payment.

- **Source**: /content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_3.txt
  **Content**: Citations (2)

References (8)

... Bank loan; It is used for reasons such as inflation, political situation, financial insufficiency, or investing. Bank loan is also important to startup, they needed a lot of money for their business expenses to stabilize the huge expenses with the less profit they are getting as a startup entrepreneurs (Patnaik, 2020). ...
Fuzzy Expert System for Bank Loan Approval
Article
Full-text available
Mar 2023
Abdullahi Ismail
Sani Ahmad
View
Show abstract
Enhancing Syndicate Lending Through AI-Powered Fine-Tuning: Leveraging LLAMA2 for Dynamic Decision Support
Conference Paper
May 2024
Ashish KumarKanika JindalHimanshu Sharma
Ankur Chaudhary
View

- **Source**: /content/drive/MyDrive/A_Projects/Model_ChatBot/ChatBot/Text_Data/File_3.txt
  **Content**: Citations (2)

References (8)

... Bank loan; It is used for reasons such as inflation, political situation, financial insufficiency, or investing. Bank loan is also important to startup, they needed a lot of money for their business expenses to stabilize the huge expenses with the less profit they are getting as a startup entrepreneurs (Patnaik, 2020). ...
Fuzzy Expert System for Bank Loan Approval
Article
Full-text available
Mar 2023
Abdullahi Ismail
Sani Ahmad
View
Show abstract
Enhancing Syndicate Lending Through AI-Powered Fine-Tuning: Leveraging LLAMA2 for Dynamic Decision Support
Conference Paper
May 2024
Ashish KumarKanika JindalHimanshu Sharma
Ankur Chaudhary
View